In [ ]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from dredFISH.Analysis import TissueGraph
from dredFISH.Visualization import viz_cell_layer
# from dredFISH.Utils.__init__plots import *  # import one of the two; but not both
from dredFISH.Utils import powerplots

# import importlib
# importlib.reload(viz_cell_layer)

In [ ]:
logging.basicConfig(format='%(asctime)s - %(message)s', 
                datefmt='%m-%d %H:%M:%S', 
                level=logging.INFO,
                )

# Path
# basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t3'
basepth = '/bigstore/GeneralStorage/Data/dredFISH/DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_v2/DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section1'

In [ ]:
dfpth = os.path.join(basepth, 'default_analysis.csv')
df = pd.read_csv(dfpth, index_col=0)
df

In [ ]:
# respth = os.path.join(basepth, 'figures')
# if not os.path.isdir(respth):
#     os.mkdir(respth)

In [ ]:
# load TMG - with cell layer obs only
TMG = TissueGraph.TissueMultiGraph(basepath=basepth, 
                                   redo=False, # load existing 
                                   quick_load_cell_obs=True,
                                  )

# spatial coordinates
layer = TMG.Layers[0]
XY = layer.XY
x, y = XY[:,0], XY[:,1]

# #### a temporary hack
# x, y = y, x 
# XY = np.vstack([x,y]).T
# ###

In [ ]:
%%time
# TMG.Geoms = list()
# TMG.add_geoms()
# geom = TMG.Geoms[0]

In [ ]:
from matplotlib.collections import LineCollection, PolyCollection, PatchCollection
from matplotlib.colors import ListedColormap

In [ ]:
c,l = pd.factorize(df['regtype_allenL2basis_k5']) #.values
region_id = c

# SG = layer.SG
# edge_list = np.asarray(SG.get_edgelist())
# assert np.all(edge_list[:,0] < edge_list[:,1])

# tmp = region_id[edge_list]
# cond = tmp[:,0]!=tmp[:,1]
# region_edge_list = edge_list[cond]
# region_edge_list = [tuple(item) for item in region_edge_list]

# segs = pd.Series(geom['line']).reindex(region_edge_list).dropna().values
# lines = LineCollection(segs, linewidths=0.5, color='lightgray')


# xyext = np.array(geom['BoundingBox'].exterior.xy).T
# mx = np.max(xyext, axis=0)
# mn = np.min(xyext, axis=0)

# p = PolyCollection(geom['poly'], 
#                    cmap=ListedColormap(['C0', 'C1', 'C2', 'C3', 'C4']), 
#                    edgecolors='none',
#                   )
# p.set_array(c)

# fig, ax = plt.subplots(figsize=(10,8))
# ax.add_collection(p) 
# # ax.add_collection(lines) #, edgecolor='none')
# ax.set_xlim([xmin, xmax])
# ax.set_ylim([ymin, ymax])
# ax.set_aspect('equal')
# ax.grid(False)
# plt.show()

In [ ]:
# from scipy.spatial import Delaunay, Voronoi
# from dredFISH.Utils.geomu import voronoi_polygons, bounding_box

In [ ]:
from scipy.spatial import Voronoi
from matplotlib.collections import PolyCollection
from matplotlib.colors import ListedColormap

In [ ]:
def prep_polygons(XY, label_ids, cmap='tab20'):
    """
    """
    unq_ids = np.unique(label_ids)
    ncolors = len(unq_ids)
    colors = sns.color_palette(cmap, ncolors)
    cmap = ListedColormap(colors)
    
    xmin, ymin = np.min(XY, axis=0)
    xmax, ymax = np.max(XY, axis=0)

    vor = Voronoi(XY, furthest_site=False)
    poly = np.array([vor.vertices[vor.regions[item]] for item in vor.point_region], dtype=object)

    # filter poly by max span
    maxspans = []
    for item in poly:
        if len(item) > 0:
            maxspan = max(np.max(item, axis=0) - np.min(item, axis=0))
        else:
            maxspan = 0
        maxspans.append(maxspan)
    maxspans = np.array(maxspans)
    th = np.percentile(maxspans, 99)
    
    # prep polygons
    p = PolyCollection(poly[maxspans<th], #[:5], #geom['poly'], 
                       cmap=cmap,
                       edgecolors='none',
                      )
    p.set_array(label_ids[maxspans<th]) #[:5])
    
    return p, (xmin, xmax, ymin, ymax)

def plot_colored_polygons(XY, c):
    """
    """
    fig, ax = plt.subplots(figsize=(10,8))
    p, bbox = prep_polygons(XY, c)
    ax.add_collection(p) 
    ax.set_xlim(bbox[:2])
    ax.set_ylim(bbox[2:])
    ax.set_aspect('equal')
    ax.grid(False)
    plt.show()

In [ ]:
%%time
c,l = pd.factorize(df['regtype_allenL2basis_k5']) #.values
plot_colored_polygons(XY, c)

# Viz

In [ ]:
%%time
# output = os.path.join(respth, 'fig1_basis_space.pdf')
output = None
powerplots.plot_basis_spatial(df, output=output)


In [ ]:
%%time
dfsub = df[df['hemi']==1]
# output = os.path.join(respth, 'fig1-2_basis_space_righthalf.pdf')
output = None
powerplots.plot_basis_spatial(dfsub, pmode='right_half', xcol='x', ycol='y', output=output)

In [ ]:
%%time
# output = os.path.join(respth, 'fig2_basis_umap.pdf')
output = None
powerplots.plot_basis_umap(df, output=output)

In [ ]:
%%time
# # resolutions = np.sort([ for col in typecols])
#     r = float(col[len("type_r"):])

typecols = df.filter(regex='^type_r', axis=1).columns
for i, col in enumerate(typecols):
    hue = col
    # output = os.path.join(respth, f'fig3-{i}_{col}.pdf')
    output = None
    powerplots.plot_type_spatial_umap(df, hue, output=output)

In [ ]:
ktypecols = df.filter(regex='^ktype_L', axis=1).columns
for i, col in enumerate(ktypecols):
    hue = col
    # output = os.path.join(respth, f'fig4-{i}_{col}.pdf')
    output = None
    powerplots.plot_type_spatial_umap(df, hue, output=output)

In [ ]:
cols = df.filter(regex='^regtype', axis=1).columns
for i, col in enumerate(cols):
    hue = col
    # output = os.path.join(respth, f'fig4-{i}_{col}.pdf')
    output = None
    powerplots.plot_type_spatial_umap(df, hue, output=output)